In [3]:
# Homework 3
if 1:
  from google.colab import drive
  #drive.flush_and_unmount()
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
 if 0:
    ! cp /content/drive/MyDrive/CaltechTinySplit.zip /content/
    ! unzip CaltechTinySplit.zip -d /content/CaltechTinySplit

In [51]:
import numpy as np
import os
import cv2 # to work with images
import glob # to load all images from a directory
import matplotlib.pyplot as plt # to show image if we need to

In [33]:
def resizeMyImage(i_image):
  i_image = cv2.cvtColor(i_image, cv2.COLOR_BGR2RGB) # (blue,green,red) -> (red,green,blue)
  i_image = cv2.resize(i_image,(128,128)) # image resized to 128x128(x3) size.
  i_image = np.resize(i_image,(1,49152)) # image is resized to 1x49152 format (vector)  
  i_image = np.concatenate((i_image,1), axis = None) # add bias
  return i_image

In [34]:
#     -- [1] TASKS --
#
# 1. read images in 128x128x3 form
# 2. convert them into 1x49152 form
#    (convert into vector format)
# 3. it will be the vector holding labels. (target vector)

str_train_dir = "/content/CaltechTinySplit/CaltechTinySplit/train/"
str_class0 = "cannon"
str_class1 = "cellphone"
#
#   Above variables are not necessary anymore.
#
str_allJpg = "/*.jpg"

image = [] # will hold one image at a time
inputs1 = [] # will hold cannon images 
inputs2 = [] # will hold cellphone images
t1 = [] # target vector for cannon
t2 = [] # target vector for cellphone

In [13]:
# cannon folder
# will run 34 times
for img in glob.glob('/content/CaltechTinySplit/CaltechTinySplit/train/cannon/*.jpg'):

  image = cv2.imread(img) # image is stored in variable "image" in 128x128x3 format
  inputs1.append(resizeMyImage(image)) # add the image at the end of inputs list
  t1.append(0)

In [35]:
# cellphone folder
# will run 47 times
for img2 in glob.glob('/content/CaltechTinySplit/CaltechTinySplit/train/cellphone/*.jpg'):

  image2 = cv2.imread(img2) # image is stored in variable "image" in 128x128x3 format
  inputs2.append(resizeMyImage(image2)) # add the image at the end of inputs list
  t2.append(1)

In [37]:
# merge both input and target values of cannon and cellphone
inputs = inputs1 + inputs2
t= t1 + t2

In [56]:
inputs = np.float16( np.array(inputs) )
targets = np.float16( np.array(t) )
print(inputs)
#print(t)

[[255. 255. 255. ... 255. 255.   1.]
 [255. 255. 255. ... 255. 255.   1.]
 [125. 131. 127. ... 227. 228.   1.]
 ...
 [255. 255. 255. ... 255. 255.   1.]
 [254. 254. 254. ... 250. 248.   1.]
 [254. 254. 254. ... 254. 254.   1.]]


In [53]:
# generate 49153 weight values
W =  np.random.rand(49153,)
W = np.float16( np.array(W) )
print(W)

[0.9893  0.0726  0.4104  ... 0.5796  0.695   0.11237]


In [43]:
# shuffle all values before training operation
from sklearn.utils import shuffle
inputs, t = shuffle(inputs, t)

In [44]:
# pre-defined functions
def tanh(x):
 return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

def softplus(x):
 return np.log(1 + np.exp(x))

def mish(x): # activation function
 return x * tanh(softplus(x))

def dmish(x):
 omega = np.exp(3*x) + 4*np.exp(2*x) + (6+4*x)*np.exp(x) + 4*(1 + x)
 delta = 1 + pow((np.exp(x) + 1), 2)
 derivative = np.exp(x) * omega / pow(delta, 2)
 return derivative

In [45]:
# train perceptron
def train_perceptron(inputs, targets, W, coef, iter):
  for j in range(0, iter):
    for i in range(0,8):
        #feed- forward
        X = inputs[i, :]
        t = targets[i]
        sum = np.dot(X,W)
        y = mish(sum)
        
        ##feed- backward
        error= t- y
        dw = coef*error*dmish(y)*X
        W = W + dw
      
    return W

In [46]:
# I have an issue in here when i'm trying to calculate new weight values.
# (I have 0 values on input arrays so it should be cause throw an RuntimeWarning error.)
# (I already tried images from different folders)

#   -- [2]  --
#    
#    > Each training sample (input) is a pair of the form <x,t> ;
#      where 
#           x is the vector of input values,
#           t is label.   (0 or 1)
#
#    > Learning rate = 0.0001
#    > Num. of iter. = 20000
#

import warnings
#suppress warnings
#warnings.filterwarnings('ignore')

print("W_old:", W)
W_new = train_perceptron(inputs, t, W, 0.0001, 20000)
print("W_new:", W_new)

W_old: [0.5835 0.8687 0.7607 ... 0.1755 0.9717 0.5273]
W_new: [nan nan nan ... nan nan nan]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
# saving new weight values
np.save('W_new.npy', W_new) # save 
weights = np.load('W_new.npy') # load

In [48]:
# test perceptron
def testPerceptron(sample_test, weights):
    y_true = []
    y_pred = []
    for i in range(0,8):
        X = inputs[i, :]
        t = targets[i]
        y_true.append(t)
        sum = np.dot(X,weights)
        y = mish(sum)
        y = np.round(y)
        y_pred.append(y)  
         
    return y_true, y_pred

In [28]:
y_true, y_pred = testPerceptron(inputs, weights)

In [29]:
# this is the true values that i calculated
print(y_true)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [30]:
# this is the predicted values that i wanted to reach
print(y_pred)

[nan, nan, nan, nan, nan, nan, nan, nan]


In [55]:
# i could not calculated these calculations because i have 0 values on input arrays
# i could not resolve that problem :(
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
acc = accuracy_score(y_true, y_pred)
conf = confusion_matrix(y_true, y_pred)
print(conf)
print(acc)

ValueError: ignored